In [ ]:
import pandas as pd
import numpy as np
import datetime
import random
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
#Load data from excel file
prod = pd.read_excel('../data/HotelReservations.xlsx',sheet_name='Reservations')

In [ ]:
#Print prod type
type(prod)

In [ ]:
#Print row x column
prod.shape

In [ ]:
#Assign pd to df
df = pd.DataFrame(prod)

In [ ]:
#Print data type of column
df.info()
pd.DataFrame(prod.dtypes, columns=['DataType'])

In [ ]:
#Find for missing data
prod.isnull().sum()

In [ ]:
#print sample data
prod.head(3)

In [ ]:
#Sort the df by year, month, date
Sort = df.sort_values(by=['arrival_year','arrival_month','arrival_date'])

In [ ]:
#Print head, mid & tail to check on first, middle & last record 
start = 1
mid = df.shape[0]//2
mid_end = mid + 1
end = 1
pd.concat([df[:start], df[mid:mid_end], df[-end:]])

In [ ]:
#Reset the index to unique id
prod.set_index('Booking_ID', inplace=True)

In [ ]:
#print index id
print(prod.index.name)

In [ ]:
prod.index

In [ ]:
#sample data aft change index
prod.head(3)

In [ ]:
#Data cleanup
#Create new field combine year, month & date
df = pd.DataFrame(prod)
df['ArrivalDate'] = pd.to_datetime(dict(year=df.arrival_year, month=df.arrival_month, day=df.arrival_date), format='%Y%m%d', errors='coerce')
print (df['ArrivalDate'].dtypes)


In [ ]:
#Data cleanup
#Create new field number of days
df['TotalDays'] = df.no_of_weekend_nights + df.no_of_week_nights
print (df['TotalDays'].dtypes)

In [ ]:
#Data cleanup
#Create new field departure date
df['DepartureDate'] = df['ArrivalDate'] + pd.to_timedelta(df['TotalDays'], unit='D')
print (df['DepartureDate'].dtypes)

In [ ]:
#Data cleanup
#Create new field booking date
df['BookingDate'] = df['ArrivalDate'] - pd.to_timedelta(df['lead_time'], unit='D')
print (df['BookingDate'].dtypes)

In [ ]:
#Data cleanup
#Create list of state in Malaysia 
State_List = ['Johor','Kedah','Kelantan','Melaka','Negeri Sembilan','Pahang','Perak','Perlis','Pulau Pinang','Selangor','Terengganu','Sabah','Sarawak','Kuala Lumpur/ Putrajaya',]


In [ ]:
#Function to assign room type
#Function with TRY and EXCEPT block to handle the error
def FunRoomType(row):
    try:
        if row['room_type_reserved'] == 'Room_Type 1':
            val = 'Standard Suite Room'
        elif row['room_type_reserved'] == 'Room_Type 2':
            val = 'Deluxe Suite Room'
        elif row['room_type_reserved'] == 'Room_Type 3':
            val = 'Super Deluxe'  
        elif row['room_type_reserved'] == 'Room_Type 4':
            val = 'Junior Suite'
        elif row['room_type_reserved'] == 'Room_Type 5':
            val = 'Suite'
        elif row['room_type_reserved'] == 'Room_Type 6':
            val = 'Executive Suite'
        elif row['room_type_reserved'] == 'Room_Type 7':
            val = 'Presidential Suite'
    except RoomTypeError:
            val = 'NA'
    return val

df['RoomType']= df.apply(FunRoomType, axis=1)

In [ ]:
#Function to assign Q1, Q2, Q3, Q4 for reporting purpose
def FunQuarter(row):
    if row['arrival_month'] <= 3:
        val = 1
    elif row['arrival_month'] <= 6:
        val = 2
    elif row['arrival_month'] <= 9:
        val = 3
    elif row['arrival_month'] <= 12:
        val = 4
    else:
        val = 0
    return val

df['Quarter']= df.apply(FunQuarter, axis=1)

In [ ]:
#Create derived column 
df['Cost']= df['avg_price_per_room']*df['TotalDays']

In [ ]:
df.head(3)

In [ ]:
#Data cleanup
#Drop raw data field as already created more derived field 

df.drop(columns=['arrival_month','arrival_date','room_type_reserved','lead_time','avg_price_per_room'], axis=1)

In [ ]:
#Create Branch Sales report & save it to Excel file

file_name = 'BranchSalesReport.xlsx'
BranchReport = pd.DataFrame(df.groupby(['arrival_year','Quarter','Branch'])["Branch"].count())
BranchReport['Percentage']= (BranchReport["Branch"]/len(df))*100
BranchReport.to_excel(file_name, engine='xlsxwriter')

In [ ]:
#Data visualization - Graph on Market Segment

MarketType = pd.DataFrame(df.groupby("market_segment_type")["market_segment_type"].count())
MarketType['market_segment_type'].plot(kind ='bar',stacked=True, figsize=(10,4), xlabel='Market Type',color=('green'))
plt.ylabel('Booking')
plt.xlabel('Market Segment')
plt.title('Market Segment Type')
plt.show()

In [ ]:
#Data visualization - Graph on Quarter vs Arrival Year

df.groupby(['Quarter', 'arrival_year']).size().unstack().plot(kind='barh',stacked=True, figsize=(10,4))
plt.ylabel('Quarter')
plt.xlabel('No of Bookings')
plt.title('Booking - Quarter vs Arrival Year')
plt.show()

In [ ]:
#Data visualization - Graph on Total days booked vs Booking Status

df.groupby(['TotalDays','booking_status']).size().unstack().plot(kind='line',stacked=True, figsize=(10,4),color=('red','green'))
plt.ylabel('Booking')
plt.xlabel('Total Days')
plt.title('Booking Status')
plt.show()

In [ ]:
#Data visualization - Graph on Branch & Year - Bookings 

df.groupby(['Branch', 'arrival_year']).size().unstack().plot(kind='bar', figsize=(15,6))
plt.ylabel('No of Room')
plt.xlabel('Room Types')
plt.title('No of Room vs Room Type')
plt.show()

In [ ]:
#Data visualization - Graph on booking per days


Arrival = pd.DataFrame(df.groupby("ArrivalDate")["ArrivalDate"].count())
Arrival['ArrivalDate'].plot(kind ='line', figsize=(10,4), xlabel='ArrivalDate', color='red')
plt.ylabel('No of Booking')
plt.xlabel('Month')
plt.title('Bookings')
plt.show()

In [ ]:
#Data visualization - Convert meal plan data to percentage

MealPlan = pd.DataFrame(df.groupby("type_of_meal_plan")["type_of_meal_plan"].count())
MealPlan['Percentage']= (MealPlan["type_of_meal_plan"]/len(df))*100

In [ ]:
#Data visualization - print meal plan 

MealPlan

In [ ]:
#Data visualization - Graph on Meal Plan percentage

MealPlan['Percentage'].plot(kind ='bar',figsize=(10,4),color="Blue")
plt.ylabel('Order Percentage')
plt.xlabel('Type of Meal')
plt.title('Meal Type')
plt.show()

In [ ]:
#Create a dataframe from selected field 

FrontDesk = (df[df['booking_status']== 'Not_Canceled'][['BookingDate','ArrivalDate','DepartureDate','Branch','TotalDays','type_of_meal_plan','Cost']])

In [ ]:
#Create excel file & file name from user input 

import os

Excel_Ext = {".xlsx"}
extension = None
while extension not in Excel_Ext:  
    filename = input("Enter Excel file name you like to save: ").lower()
    extension = os.path.splitext(filename)[1]
FrontDesk.to_excel(filename, engine='xlsxwriter')
print('Saved successfull!!')